In [ ]:
#import the library
import scanpy as sc
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb
from gprofiler import GProfiler
import seaborn as sns
import rpy2.rinterface_lib.callbacks
import logging
import tensorflow as tf
import scipy.sparse
import os

from rpy2.robjects import pandas2ri
import anndata2ri

import importlib
import warnings
warnings.filterwarnings("ignore")
import pickle as pkl
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
values = [0,1]
colors = [(227, 227, 227), (255, 42, 18)]
norm = plt.Normalize(min(values), max(values))
my_cmap = LinearSegmentedColormap.from_list(
    '', [(norm(value), tuple(np.array(color) / 255)) for value, color in zip(values, colors)])

In [ ]:
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

In [ ]:
plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
sc.logging.print_versions()

In [ ]:
%%R
.libPaths(.libPaths('win-library\\4.3'))

library(scran)
library(Seurat)
library(RColorBrewer)
library(slingshot)
library(monocle)
library(gam)
library(ggplot2)
library(plyr)
library(MAST)
library(clusterExperiment)
library(monocle3)
library(SeuratWrappers)
library(magrittr)
library(dplyr)

In [ ]:
metadata = pd.read_csv("GSE200997_GEO_processed_CRC_10X_cell_annotation.csv",index_col = 0)

In [ ]:
tp = pd.read_csv("GSE200997_GEO_processed_CRC_10X_raw_UMI_count_matrix.csv",index_col = 0, iterator=True, chunksize=1000)
df = pd.concat(tp, ignore_index=True)

In [ ]:
var_names = pd.read_csv("GSE200997_GEO_processed_CRC_10X_raw_UMI_count_matrix.csv",usecols = [0]) 

In [ ]:
obs_names = pd.read_csv("GSE200997_GEO_processed_CRC_10X_raw_UMI_count_matrix.csv", nrows = 1) 

In [ ]:
df.index =  var_names.values.ravel()

In [ ]:
adata = anndata.AnnData(df.T)

In [ ]:
adata.obs = metadata

In [ ]:
adata.write('raw.h5ad')

In [ ]:
adata = sc.read_h5ad('raw.h5ad')

In [ ]:
adata_17 = sc.read_h5ad('GSE200997_T17.h5ad')

In [ ]:
adata_17_raw = adata[adata_17.obs_names,:]

In [ ]:
colnames = adata_17_raw.obs_names
rownames = adata_17_raw.var_names
counts = adata_17_raw.X.T

In [ ]:
%%R -i colnames -i rownames -i counts
colnames(counts) = colnames
rownames(counts) = rownames

srat <- CreateSeuratObject(counts = counts, project = "GSE200997", min.cells = 0, min.features = 0, assay = "RNA")

In [ ]:
%R saveRDS(srat, 'F:/CRC/AA_Done/GSE200997_T17.rds')

In [ ]:
adata.obs['n_counts'] = adata.X.sum(1)
adata.obs['log_counts'] = np.log(adata.obs['n_counts'])
adata.obs['n_genes'] = (adata.X > 0).sum(1)
adata.obs['log10GenesPerUMI'] = np.log10(adata.obs['n_genes'])/np.log10(adata.obs['n_counts']) #This is the complexity

mt_gene_mask = [gene.startswith('MT-') for gene in adata.var_names]
adata.obs['mt_frac'] = np.array(adata.X[:, mt_gene_mask].sum(1).ravel())[0]/adata.obs['n_counts']
ribo_gene_mask = [gene.startswith('RPL') or gene.startswith('RPS') for gene in adata.var_names]
adata.obs['ribo_frac'] = np.array(adata.X[:, ribo_gene_mask].sum(1).ravel())[0]/adata.obs['n_counts']

In [ ]:
# Filter cells according to identified QC thresholds:
print('Total number of cells: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, min_counts = 500)
print('Number of cells after min count filter: {:d}'.format(adata.n_obs))


adata = adata[adata.obs['mt_frac'] < 0.2]
print('Number of cells after MT filter: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, min_genes = 200)
print('Number of cells after gene filter: {:d}'.format(adata.n_obs))

In [ ]:
import doubletdetection
clf = doubletdetection.BoostClassifier(n_components = 50, n_top_var_genes = 3000)
# raw_counts is a cells by genes count matrix
labels = clf.fit(adata.X).predict()
# higher means more likely to be doublet
scores = clf.doublet_score()

In [ ]:
print('Number of cells before DP filter: {:d}'.format(adata.n_obs))
adata = adata[labels == 0]
print('Number of cells after DP filter: {:d}'.format(adata.n_obs))

In [ ]:
sc.pp.normalize_total(adata, key_added = 'normalization_factors')
sc.pp.log1p(adata)

In [ ]:
sc.pp.highly_variable_genes(potential_17, flavor='seurat', n_top_genes=3000)
sc.pp.pca(potential_17, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(potential_17, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(potential_17, resolution = 1, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(potential_17)

In [ ]:
%matplotlib inline
sc.pl.tsne(potential_17, color=['leiden','RORC','IL17A','IL17F'],size = 180,cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(adata, restrict_to = ('leiden', ['3']), resolution = 0.3, key_added= 'leiden1')
sc.pl.tsne(adata, color=['leiden1','RORC','IL17A','IL17F'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
potential_17 = adata[adata.obs['leiden'].isin(['6']),:]

In [ ]:
del potential_17.uns

In [ ]:
sc.pp.highly_variable_genes(potential_17, flavor='seurat', n_top_genes=3000)
sc.pp.pca(potential_17, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(potential_17, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(potential_17, resolution = 1, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(potential_17)

In [ ]:
%matplotlib inline
sc.pl.tsne(potential_17, color=['leiden','RORC','IL17A','IL17F'],size = 180,cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden', ['5']), resolution = 0.6, key_added= 'leiden1')
sc.pl.tsne(potential_17, color=['leiden1','RORC','IL17A','IL17F'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden1', ['10']), resolution = 0.4, key_added= 'leiden2')
sc.pl.tsne(potential_17, color=['leiden2','RORC','IL17A','IL17F'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden2', ['7']), resolution = 0.5, key_added= 'leiden3')
sc.pl.tsne(potential_17, color=['leiden3','RORC','IL17A','IL17F'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
IL17_genesmask = [gene.startswith("IL17") and gene.startswith("IL17R")==0 for gene in potential_17.var_names]
IL17_genes = potential_17.var_names[IL17_genesmask]
IL17_exp_set ={
    'cytokine': IL17_genes, 'TF': 'RORC'
}
sc.pl.dotplot(potential_17,IL17_exp_set,groupby = 'leiden' , vmax = 1, swap_axes = False, dot_min =0.1, dot_max =1,standard_scale = 'var')

In [ ]:
T17_1 = potential_17[potential_17.obs['leiden'].isin(['2','5','6']),:]

In [ ]:
T17_2 = adata[adata.obs['leiden1'].isin(['3,1','3,2','33']),:]

In [ ]:
#Why all of a sudden this does not work??
adata_17 = T17_1.concatenate(T17_2, batch_key = 'original_cluster', batch_categories=['6','3-33'],join = 'outer',fill_value=0)

In [ ]:
sc.pp.highly_variable_genes(adata_17, flavor='seurat', n_top_genes=3000)
sc.pp.pca(adata_17, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(adata_17, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(adata_17, resolution = 1, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(adata_17)

In [ ]:
%matplotlib inline
sc.pl.tsne(adata_17, color=['leiden','RORC','IL17A','IL17F'],size = 40,cmap = my_cmap)

In [ ]:
adata_17.write('GSE200997_T17.h5ad')

In [ ]:
adata.write('GSE200997_processed.h5ad')

In [ ]:
adata = sc.read_h5ad('GSE200997_processed.h5ad')

In [ ]:
bioinfo = pd.read_csv('GSE200997_bioinfo.csv')

In [ ]:
def convert_tnm_to_stage(t, n, m):
    """
    Convert TNM components to an overall cancer stage.
    
    Parameters:
    t (str): Tumor size and extent (e.g., 'Tis', 'T1', 'T2', 'T3', 'T4').
    n (str): Node involvement (e.g., 'N0', 'N1', 'N2', 'N3').
    m (str): Metastasis presence (e.g., 'M0', 'M1').
    
    Returns:
    int: Overall cancer stage (0 to IV).
    """
    t = t[0].lower()
    n = n[0].lower()
    m = m[0].lower()
    # Handling metastasis first because it overrides other categories
    if m == '1':
        return 'IV'
    if t.isnumeric()==0 or n.isnumeric()==0:
        return 'N/A'
    
    # Mapping T and N to stages
    if t == '0' and n == '0' and m != '1':
        return '0'
    elif t == '1' and n == '0' and m != '1':
        return 'I'
    elif (t == '2' and n == '0' and m != '1') or (t == '1' and n == '1' and m != '1'):
        return 'II'
    elif (t == '2' and n == '1' and m != '1') or (t in ['3'] and n in ['0', '1'] and m != '1'):
        return 'III'
    
    # Default to highest stage if other conditions are not met (typically not used, more complex logic needed in real cases)
    return 'IV'

In [ ]:
patient = [i.split('_')[1].split('cac')[1] for i in adata.obs_names]

In [ ]:
adata.obs['samples'] = [i.upper() for i in adata.obs['samples']]

In [ ]:
adata.obs['patient'] = patient

In [ ]:
bioinfo['patient'] = [i.split('CAC')[1] for i in bioinfo['SAMPLE_ID']]
site_dict = dict(zip(bioinfo['patient'], bioinfo['Location2']))
site = [site_dict[i] for i in adata.obs['patient']]

In [ ]:
gender_dict = dict(zip(bioinfo['patient'], bioinfo['Gender']))
gender = [gender_dict[i] for i in adata.obs['patient']]

In [ ]:
age_dict = dict(zip(bioinfo['patient'], bioinfo['Age']))
age = [age_dict[i] for i in adata.obs['patient']]

In [ ]:
bioinfo['TNM_T'] = [i.split('T')[1].split('N')[0] for i in bioinfo['Stage']]
bioinfo['TNM_N'] = [i.split('N')[1].split('M')[0] for i in bioinfo['Stage']]
bioinfo['TNM_M'] = '0'
bioinfo['roman stage'] = 'N/A'
for i,stage in enumerate(bioinfo['Stage']):
    if len(stage.split('M')) > 1:
        bioinfo['TNM_M'][i] = stage.split('M')[1]
    bioinfo['roman stage'][i] = convert_tnm_to_stage(bioinfo['TNM_T'][i], bioinfo['TNM_N'][i], bioinfo['TNM_M'][i])

In [ ]:
bioinfo

In [ ]:
for key in ['T','N','M']:
    colname = 'TNM_'+key
    adata.obs[colname] = 'N/A'
    T_dict = dict(zip(bioinfo['SAMPLE_ID'], bioinfo[colname]))
    for i,sid in enumerate(adata.obs['samples']):
        if sid.startswith('T'):
            adata.obs[colname][i] = T_dict[sid] 

In [ ]:
R_stage_dict =  dict(zip(bioinfo['SAMPLE_ID'], bioinfo['roman stage']))
adata.obs['roman stage'] = 'N/A'
for i,sid in enumerate(adata.obs['samples']):
    if sid.startswith('T'):
        adata.obs['roman stage'][i] = R_stage_dict[sid] 

In [ ]:
adata.obs['tissue'] = 'Colon ' + adata.obs['Condition'].astype(str).map(str)

In [ ]:
adata.obs['patient'] = patient
adata.obs['gender'] = gender
adata.obs['age'] = age
adata.obs['site'] = site
adata.obs['stage'] = adata.obs['roman stage']

In [ ]:
adata.obs['gender'] = [i[0].upper()+i[1:] for i in adata.obs['gender']]

In [ ]:
sc.pl.tsne(adata,color = ['patient','tissue','gender','age','site','TNM_T','TNM_N','TNM_M','roman stage'])

In [ ]:
T_adata = adata[adata.obs['leiden'].isin([str(i) for i in [0, 1, 2, 6, 4 ,7, 19, 14, 24]]),:]

In [ ]:
sc.pp.highly_variable_genes(T_adata, flavor='seurat', n_top_genes=3000)
sc.pp.pca(T_adata, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(T_adata, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(T_adata, resolution = 0.8, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(T_adata)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(T_adata, restrict_to = ('leiden', ['0']), resolution = 0.9, key_added= 'leiden1')
sc.pl.tsne(T_adata, color=['leiden1','CD3E','CD3G','CD3D','TRDC'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
%matplotlib inline
sc.pl.tsne(T_adata, color=['leiden','CD3E','CD3G','CD3D','TRDC','ICOS','CD4','CD8A','CD8B','FOXP3','IFNG','TRGV4','TRBV7-4'],cmap = my_cmap)

In [ ]:
gamma_genesmask = [gene.startswith("TRG") for gene in T_adata.var_names]
gamma_genes = T_adata.var_names[gamma_genesmask]
delta_genesmask = [gene.startswith("TRD") for gene in T_adata.var_names]
delta_genes = T_adata.var_names[delta_genesmask]

In [ ]:
TCR_exp_set ={
"gamma": gamma_genes, "delta": delta_genes, "CD3S": ['CD3E','CD3D','CD3G']
}
sc.pl.dotplot(T_adata,TCR_exp_set,groupby = 'leiden1' , vmax = 1, swap_axes = False, dot_min =0, dot_max =1,standard_scale = 'var')

In [ ]:
keep_list = ['0,1','0,4','0,6','0,7','0,9','0,10']

In [ ]:
gd_adata = T_adata[T_adata.obs['leiden1'].isin(keep_list),:]

In [ ]:
gd_adata.write('gdT.h5ad')

## Do they show any sign of IL17 secreting?

In [ ]:
sc.pl.tsne(gd_adata,color = ['IL17A','RORC'], cmap = my_cmap)

In [ ]:
TCR_exp_set ={
"gamma": gamma_genes, "delta": delta_genes, "CD3S": ['CD3E','CD3D','CD3G']
}
sc.pl.dotplot(gd_adata,TCR_exp_set,groupby = 'leiden1' , vmax = 1, swap_axes = False, dot_min =0, dot_max =1,standard_scale = 'var')

In [ ]:
adata_gd = sc.read_h5ad('gdT.h5ad')

In [ ]:
bioinfo = pd.read_csv('bioinfo.csv')

In [ ]:
patient = [i.split('_')[1].split('cac')[1] for i in adata_gd.obs_names]

In [ ]:
adata_gd.obs['samples'] = [i.upper() for i in adata_gd.obs['samples']]

In [ ]:
adata_gd.obs['patient'] = patient

In [ ]:
bioinfo['patient'] = [i.split('CAC')[1] for i in bioinfo['SAMPLE_ID']]
site_dict = dict(zip(bioinfo['patient'], bioinfo['Location2']))
site = [site_dict[i] for i in adata_gd.obs['patient']]

In [ ]:
gender_dict = dict(zip(bioinfo['patient'], bioinfo['Gender']))
gender = [gender_dict[i] for i in adata_gd.obs['patient']]

In [ ]:
age_dict = dict(zip(bioinfo['patient'], bioinfo['Age']))
age = [age_dict[i] for i in adata_gd.obs['patient']]

In [ ]:
bioinfo['TNM_T'] = [i.split('T')[1].split('N')[0] for i in bioinfo['Stage']]
bioinfo['TNM_N'] = [i.split('N')[1].split('M')[0] for i in bioinfo['Stage']]
bioinfo['TNM_M'] = '0'
bioinfo['roman stage'] = 'N/A'
for i,stage in enumerate(bioinfo['Stage']):
    if len(stage.split('M')) > 1:
        bioinfo['TNM_M'][i] = stage.split('M')[1]
    bioinfo['roman stage'][i] = convert_tnm_to_stage(bioinfo['TNM_T'][i], bioinfo['TNM_N'][i], bioinfo['TNM_M'][i])

In [ ]:
def convert_tnm_to_stage(t, n, m):
    """
    Convert TNM components to an overall cancer stage.
    
    Parameters:
    t (str): Tumor size and extent (e.g., 'Tis', 'T1', 'T2', 'T3', 'T4').
    n (str): Node involvement (e.g., 'N0', 'N1', 'N2', 'N3').
    m (str): Metastasis presence (e.g., 'M0', 'M1').
    
    Returns:
    int: Overall cancer stage (0 to IV).
    """
    t = t[0].lower()
    n = n[0].lower()
    m = m[0].lower()
    # Handling metastasis first because it overrides other categories
    if m == '1':
        return 'IV'
    if t.isnumeric()==0 or n.isnumeric()==0:
        return 'N/A'
    
    # Mapping T and N to stages
    if t == '0' and n == '0' and m != '1':
        return '0'
    elif t == '1' and n == '0' and m != '1':
        return 'I'
    elif (t == '2' and n == '0' and m != '1') or (t == '1' and n == '1' and m != '1'):
        return 'II'
    elif (t == '2' and n == '1' and m != '1') or (t in ['3'] and n in ['0', '1'] and m != '1'):
        return 'III'
    
    # Default to highest stage if other conditions are not met (typically not used, more complex logic needed in real cases)
    return 'IV'

In [ ]:
for key in ['T','N','M']:
    colname = 'TNM_'+key
    adata_gd.obs[colname] = 'N/A'
    T_dict = dict(zip(bioinfo['SAMPLE_ID'], bioinfo[colname]))
    for i,sid in enumerate(adata_gd.obs['samples']):
        if sid.startswith('T'):
            adata_gd.obs[colname][i] = T_dict[sid] 

In [ ]:
R_stage_dict =  dict(zip(bioinfo['SAMPLE_ID'], bioinfo['roman stage']))
adata_gd.obs['roman stage'] = 'N/A'
for i,sid in enumerate(adata_gd.obs['samples']):
    if sid.startswith('T'):
        adata_gd.obs['roman stage'][i] = R_stage_dict[sid] 

In [ ]:
adata_gd.obs['tissue'] = 'Colon ' + adata_gd.obs['Condition'].astype(str).map(str)

In [ ]:
adata_gd.obs['patient'] = patient
adata_gd.obs['gender'] = gender
adata_gd.obs['age'] = age
adata_gd.obs['site'] = site
adata_gd.obs['stage'] = adata_gd.obs['roman stage']

In [ ]:
adata_gd.obs['gender'] = [i[0].upper()+i[1:] for i in adata_gd.obs['gender']]

In [ ]:
sc.pl.tsne(adata_gd,color = ['patient','tissue','gender','age','site','TNM_T','TNM_N','TNM_M','roman stage'])

In [ ]:
adata_gd.obs[['patient','tissue','gender','age','site','TNM_T','TNM_N','TNM_M','stage']].to_csv('F:/CRC/AA_Done/GSE200997_gd.csv')

In [ ]:
adata_gd.write('GSE200997_gd.h5ad')

In [ ]:
gd_info = pd.read_csv('F:/CRC/AA_Done/GSE200997_gd.csv')

In [ ]:
adata.var_names_make_unique()

In [ ]:
counts = adata[gd_info['Unnamed: 0'],:].X.T
colnames = gd_info['Unnamed: 0']
rownames = adata.var_names

In [ ]:
%%R -i colnames -i rownames -i counts 

colnames(counts) = colnames
rownames(counts) = rownames

srat <- CreateSeuratObject(counts = counts, project = "GSE200997", min.cells = 0, min.features = 0, assay = "RNA")

In [ ]:
%%R -i gd_info
srat = AddMetaData(srat, metadata = gd_info)
saveRDS(srat, 'F:/CRC/AA_Done/GSE200997_gd.rds')

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
values = [0,1]
colors = [(227, 227, 227), (255, 42, 18)]
norm = plt.Normalize(min(values), max(values))
my_cmap = LinearSegmentedColormap.from_list(
    '', [(norm(value), tuple(np.array(color) / 255)) for value, color in zip(values, colors)])

In [ ]:
adata = sc.read_h5ad('GSE200997_processed.h5ad')
adata_17 = sc.read_h5ad('GSE200997_T17.h5ad')
adata_gd = sc.read_h5ad('GSE200997_gd.h5ad')

In [ ]:
adata.obs['IL17 secreting selected'] = '0'
adata.obs['IL17 secreting selected'][adata.obs_names.isin(adata_17.obs_names)] = '1'

In [ ]:
adata.obs['gdT selected'] = '0'
adata.obs['gdT selected'][adata.obs_names.isin(adata_gd.obs_names)] = '1'

In [ ]:
plt.close()
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['figure.figsize'] = [8,8]
fig = sc.pl.tsne(adata, color=['RORC','IL17A','IL17F','IL17 secreting selected'],
                 size =5, ncols = 2, palette = ['#E3E3E3', '#FF2A12'], cmap = my_cmap, return_fig = True, legend_fontsize = 'large')
ax = fig.get_axes()
for i in range(0,len(ax)):
    ax[i].xaxis.label.set_fontsize(22)
    ax[i].xaxis.label.set_fontweight('bold')
    ax[i].yaxis.label.set_fontsize(22)
    ax[i].title.set_fontsize(30)
    ax[i].yaxis.label.set_fontweight('bold')
    ax[i].title.set_fontweight('bold')
fig.savefig('17_selected.png',dpi = 300,bbox_inches='tight') 

In [ ]:
plt.close()
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['figure.figsize'] = [8,8]
fig = sc.pl.tsne(adata, color=['CD3E','CD3D','CD3G','CD247','TRDC','gdT selected'],
                 size =5, ncols = 2, palette = ['#E3E3E3', '#FF2A12'], cmap = my_cmap, return_fig = True, legend_fontsize = 'large')
ax = fig.get_axes()
for i in range(0,len(ax)):
    ax[i].xaxis.label.set_fontsize(22)
    ax[i].xaxis.label.set_fontweight('bold')
    ax[i].yaxis.label.set_fontsize(22)
    ax[i].title.set_fontsize(30)
    ax[i].yaxis.label.set_fontweight('bold')
    ax[i].title.set_fontweight('bold')
fig.savefig('gd_selected.png',dpi = 300,bbox_inches='tight') 